<a href="https://colab.research.google.com/github/Damilola-ops/Text-de-shuffling/blob/main/Text_de_shuffling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install pandas 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.5 MB/s 
     |████████████████████████████████| 365 kB 66.8 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.9 MB/s 
     |████████████████████████████████| 596 kB 56.8 MB/s 
     |████████████████████████████████| 141 kB 63.3 MB/s 
     |████████████████████████████████| 212 kB 66.0 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [2]:
from transformers import EncoderDecoderModel, EncoderDecoderConfig,Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartConfig,BartTokenizerFast,BartForCausalLM,BartModel
import torch
import pandas as pd 
import numpy as np
import sklearn.preprocessing 
from torch.nn import functional as F 
import datasets
from datasets import load_dataset


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/train (8).csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv')

val_df = pd.read_csv('/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv')

In [5]:
#using huggingface's dataset function to load multiple datasets at once 
dataset = load_dataset( 'csv',data_files = {'train':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/train (8).csv'],
                                         'val':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv'],
                                         'test':['/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv']})

Using custom data configuration default-f3bbdea8f3573fa9


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f3bbdea8f3573fa9/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
#huggingface's dataset returns a datasetdict object of our data
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 40001
    })
    val: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label', 'id'],
        num_rows: 4001
    })
})

In [6]:
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

In [8]:
text = 'offisong is a fool '
tokens = tokenizer(text)
tokens

{'input_ids': [0, 1529, 354, 1657, 16, 10, 17275, 1437, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
#testing out our tokenizer
sample = train_df['label'].loc[5]
tokens = tokenizer.tokenize(sample)
token_id = tokenizer.convert_tokens_to_ids(tokens)
print('sample text: ', sample)
print('tokens : ',tokens)
print('token_id : ',token_id)

sample text:  Surprisingly, the transition is fastest with an intermediate flexibility of the hinge.
tokens :  ['Sur', 'prisingly', ',', 'Ġthe', 'Ġtransition', 'Ġis', 'Ġfastest', 'Ġwith', 'Ġan', 'Ġintermediate', 'Ġflexibility', 'Ġof', 'Ġthe', 'Ġhinge', '.']
token_id :  [27526, 47607, 6, 5, 3868, 16, 6273, 19, 41, 21398, 8243, 9, 5, 33503, 4]


In [13]:
original_text = tokenizer.decode(token_id)
original_text

'Surprisingly, the transition is fastest with an intermediate flexibility of the hinge.'

In [14]:
#a function to tokenize all the rows in our dataset
max_text_length = 40 
max_label_length = 40 
def preprocess_function(sample):
    # Getting text and label
    text = sample["text"]
    label = sample["label"]
    # Tokenizing the text and label
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_text_length)
    outputs = tokenizer(label, padding="max_length", truncation=True, max_length=max_label_length)
    sample["input_ids"] = inputs.input_ids
    sample["attention_mask"] = inputs.attention_mask
    sample["decoder_input_ids"] = outputs.input_ids
    sample["decoder_attention_mask"] = outputs.attention_mask
    sample["labels"] = outputs.input_ids
    # The labels are used to calcuate the loss while training, and because we added padding to make all tokens to be of same size,
    # we also need to convert the padding number ( 0 ) to ( -100 ), so that we can tell huggingface that these number can be ignorned while calcuating loss. 
    # Why specifically -100 ? It's simply an arbitrary number, again so that huggingface can ignore this number while calcuating loss
    sample["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in sample["labels"]]

    return sample

In [15]:
batch = 16
tokenized_dataset=dataset.map(preprocess_function,batch_size=batch,batched=True)

  0%|          | 0/2501 [00:00<?, ?ba/s]

  0%|          | 0/625 [00:00<?, ?ba/s]

  0%|          | 0/251 [00:00<?, ?ba/s]

In [16]:
tokenized_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [24]:
model = BartForCausalLM.from_pretrained('facebook/bart-base')

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gr

In [25]:
model#model architecture

BartForCausalLM(
  (model): BartDecoderWrapper(
    (decoder): BartDecoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartDecoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): GELUActivation()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, 

In [26]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id =tokenizer.pad_token_id
model.config.vocab_size = model.config.vocab_size


In [31]:
N_epochs = 3
args = Seq2SeqTrainingArguments(
    "Scambled Text",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch,
    per_device_eval_batch_size=batch,
    num_train_epochs=N_epochs,
    fp16=True, # This will help in increasing the speed of training
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
trainer = Seq2SeqTrainer(model = model,
                         args = args,
                         train_dataset = tokenized_dataset['train'],
                         eval_dataset = tokenized_dataset['val']
                        )

                                            
                         
                         

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForCausalLM.forward` and have been ignored: decoder_attention_mask, decoder_input_ids, id, text. If decoder_attention_mask, decoder_input_ids, id, text are not expected by `BartForCausalLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40001
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7503


Epoch,Training Loss,Validation Loss


Saving model checkpoint to Scambled Text/checkpoint-500
Configuration saved in Scambled Text/checkpoint-500/config.json
Model weights saved in Scambled Text/checkpoint-500/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-1000
Configuration saved in Scambled Text/checkpoint-1000/config.json
Model weights saved in Scambled Text/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-1500
Configuration saved in Scambled Text/checkpoint-1500/config.json
Model weights saved in Scambled Text/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to Scambled Text/checkpoint-2000
Configuration saved in Scambled Text/checkpoint-2000/config.json
Model weights saved in Scambled Text/checkpoint-2000/pytorch_model.bin


In [ ]:
def generate_predictions(batch):

    # Tokenizing the test
    inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=max_text_length, return_tensors="pt")
    
    # Sending the tensors to GPU
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # Generating the predicted tokens ids
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # Converting the token ids to sentence
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["predictions"] = output_str

    return batch
    
    
    

In [ ]:
#testing out the model on a sample instance 


In [ ]:
test_text = train_df['text'].loc[25]
expected_output = train_df['label'].loc[25]
input = tokenizer(test_text,return_tensors= 'pt')
input_id = tokenizer.convert_tokens_to_ids(input)
attention_mask = input.attention_mask
output = model.generate(**input.to('cuda'))
output = tokenizer.decode(output[0])


RuntimeError: ignored

In [ ]:
test_input = bart_tokenizer(test_text,return_tensors='pt')
output = model(**test_input.to('cuda'))
output = bart_tokenizer.decode(output)

In [ ]:
output

In [ ]:
test_input

{'input_ids': tensor([[    0,  7068,   281, 12203,    10, 26436,   811, 28262,   152,  6609,
           634,  9158,    12,  3341,     8,     5,  2225, 36173,     4, 27562,
         30985,     9,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
input_id

[3, 3]

In [ ]:
bart_tokenizer.decode(output[0])

'<s>Most vascular Hessian-based enhancing techniques often well-known rely filters. on</s></s>'

In [ ]:
test_text

'feasibility a Hessian deficiencies This structures using curve-like and the paper matrix. investigates detecting of'

In [ ]:
test_df.head(5)

,id,text,label
0,0,safely objects move. that system images detect...,"system Using approach, move. safely skip of th..."
1,1,We detectors popular influence of confidences ...,confidences popular different in detectors We ...
2,2,compact coding present supervised We a approac...,"We a supervised approach, compact present codi..."
3,3,study high-throughput vital of quantitative be...,is for and individuals study of of collective ...
4,4,on data sets. We evaluate method many challeng...,sets. challenging the data We method on many e...
